# Knowledge Graph extraction from documents
Based on https://github.com/lamm-mit/GraphReasoning

This notebook creates a knowledge graph using functions in the GraphReasoning_Mod module which is adpated from https://github.com/lamm-mit/GraphReasoning to work with huggingface endpoints and for use with huggingface documents. 
- documents are scrapped using data.ipynb from pdfs, blogs, and youtube videos transcripts. The only metadata used in the knowledge graph creating is adding relationships between source document and node.
- LLM model Mistral-Nemo-Instruct-2407
- Embedding model: dunzhang/stella_en_1.5B_v5 (chosen based on size and position on leaderboard DEC 2024)
- KG created with refinement loops in when identifying nodes. 
- KG uses simplify graph and additonal tools in GraphReasoning_Mod

## Setup

In [9]:
%pip install -r requirements.txt -q

Note: you may need to restart the kernel to use updated packages.


In [10]:
import GraphReasoning_Mod
import importlib

In [11]:
# Reload the module to reflect any changes
importlib.reload(GraphReasoning_Mod)

# Re-import all objects from the module
from GraphReasoning_Mod import *

In [12]:
from langchain_huggingface import HuggingFaceEndpoint

#Initialize the model endpoint
HOST_URL_INF = ":8080"
MAX_NEW_TOKENS = 1500

TEMPERATURE = 0.2
TIMEOUT = 180
TOP_P = .9

llm = HuggingFaceEndpoint(
    endpoint_url=HOST_URL_INF,
    task="text-generation",
    max_new_tokens=MAX_NEW_TOKENS,
    do_sample=False,
    temperature = TEMPERATURE,
    timeout=TIMEOUT,
    top_p=TOP_P
)
#print(llm.invoke("What is HuggingFace?"))

In [13]:
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "dunzhang/stella_en_1.5B_v5" #"BAAI/bge-small-en-v1.5" #dunzhang/stella_en_1.5B_v5
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
embd = HuggingFaceEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

## Import Documents

In [6]:
import pickle
import os

# load pickled documents
pickle_file_path = 'data/storage/full_all_documents.pkl'
if os.path.exists(pickle_file_path):
    with open(pickle_file_path, 'rb') as f:
        all_pdf_docs, all_yt_docs, all_blog_docs = pickle.load(f)

#check if the documents are loaded
print("Number of PDF documents:", len(all_pdf_docs))
print("Number of YouTube documents:", len(all_yt_docs))
print("Number of blog documents:", len(all_blog_docs))

#print dictionary keys from metadata
print("Metadata keys for PDF documents:", all_pdf_docs[0].metadata.keys())
print("Metadata keys for yt documents:", all_yt_docs[0].metadata.keys())
print("Metadata keys for blog documents:", all_blog_docs[0].metadata.keys())

# Combine all documents into a single list
all_docs = all_pdf_docs+  all_yt_docs+  all_blog_docs

print(f"Total number of documents: {len(all_docs)}")


Number of PDF documents: 2048
Number of YouTube documents: 442
Number of blog documents: 11
Metadata keys for PDF documents: dict_keys(['source', 'page'])
Metadata keys for yt documents: dict_keys(['source', 'title', 'description', 'upload_date', 'duration', 'uploader', 'view_count', 'like_count', 'dislike_count', 'categories', 'tags', 'sequence'])
Metadata keys for blog documents: dict_keys(['source', 'title', 'description', 'language'])
Total number of documents: 2501


## Create create and save networkx graph

In [7]:

# Initialize variables
G_existing = None
existing_node_embeddings = None
failed_batches = []  
output_directory = 'data_w_refine'


# Process documents in chunks
chunk_size = 1500
batch_size = 10

# Split all_docs into batches of size batch_size
doc_batches = [all_docs[i:i + batch_size] for i in range(0, len(all_docs), batch_size)]

for batch_idx, doc_batch in tqdm(enumerate(doc_batches), total=len(doc_batches), desc="Processing batches..."):
    try:
        G_existing, existing_node_embeddings, res = add_new_subgraph_from_docs(
            input_docs=doc_batch,
            llm=llm,
            embd=embd,
            data_dir_output=f"./{output_directory}/",
            verbatim=False,
            size_threshold=10,
            chunk_size=chunk_size,
            do_Louvain_on_new_graph=True,
            include_contextual_proximity=False,
            repeat_refine=2,
            similarity_threshold=0.95,
            do_simplify_graph=True,
            return_only_giant_component=False,
            save_common_graph=False,
            G_exisiting=G_existing,
            graph_GraphML_exisiting=None,
            existing_node_embeddings=existing_node_embeddings
        )

        print(f"Processed batch {batch_idx}, updated graph stats:", res)
        with open(f'{output_directory}/embeddings.pkl', 'wb') as f:
            pickle.dump(existing_node_embeddings, f)
        with open(f'{output_directory}/failed_batches.pkl', 'wb') as f:
            pickle.dump(failed_batches, f)
    except Exception as e:
        # Log the failed batch index
        failed_batches.append(batch_idx)
        print(f"Error processing batch {batch_idx} with batch size {batch_size}: {e}")

# Final graph statistics and saving
print("Final graph statistics:", res if 'res' in locals() else "No successful batches")
print("Failed batch indices:", failed_batches)

Processing batches...:   0%|          | 1/251 [10:06<42:06:55, 606.46s/it]

Processed batch 0, updated graph stats: ({'Number of Nodes': 28, 'Number of Edges': 70, 'Average Degree': 5.0, 'Density': 0.18518518518518517, 'Connected Components': 1, 'Number of Communities': 5}, False)


Processing batches...:   1%|          | 2/251 [23:44<50:32:41, 730.77s/it]

Processed batch 1, updated graph stats: ({'Number of Nodes': 55, 'Number of Edges': 120, 'Average Degree': 4.363636363636363, 'Density': 0.08080808080808081, 'Connected Components': 1, 'Number of Communities': 7}, False)


Processing batches...:   1%|          | 3/251 [35:58<50:26:39, 732.26s/it]

...

Error occurred: (MaxRetryError('HTTPConnectionPool(host=\'trac-crimson.ern.nps.edu\', port=8090): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x000001C3C2F28170>: Failed to resolve \'trac-crimson.ern.nps.edu\' ([Errno 11001] getaddrinfo failed)"))'), '(Request ID: 405eec9e-dab8-43a8-93b9-b7a3dddbb46c)')
Processed batch 239, updated graph stats: None
Error occurred: (MaxRetryError('HTTPConnectionPool(host=\'trac-crimson.ern.nps.edu\', port=8090): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x000001C3C2F29F70>: Failed to resolve \'trac-crimson.ern.nps.edu\' ([Errno 11001] getaddrinfo failed)"))'), '(Request ID: 389a6e91-34ed-472f-a50f-7ffb24e79c3d)')
Processed batch 240, updated graph stats: None


Traceback (most recent call last):
  File "c:\Users\jonathan.kasprisin\AppData\Local\miniconda3\envs\kg_ilp\Lib\site-packages\urllib3\connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jonathan.kasprisin\AppData\Local\miniconda3\envs\kg_ilp\Lib\site-packages\urllib3\util\connection.py", line 60, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jonathan.kasprisin\AppData\Local\miniconda3\envs\kg_ilp\Lib\socket.py", line 976, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
socket.gaierror: [Errno 11001] getaddrinfo failed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\jonathan

Error occurred: (MaxRetryError('HTTPConnectionPool(host=\'trac-crimson.ern.nps.edu\', port=8090): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x000001C3BECF7B30>: Failed to resolve \'trac-crimson.ern.nps.edu\' ([Errno 11001] getaddrinfo failed)"))'), '(Request ID: 1c6666df-1fd1-4188-8964-06974738b56c)')
Processed batch 241, updated graph stats: None
Error occurred: (MaxRetryError('HTTPConnectionPool(host=\'trac-crimson.ern.nps.edu\', port=8090): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x000001C3C2F2B230>: Failed to resolve \'trac-crimson.ern.nps.edu\' ([Errno 11001] getaddrinfo failed)"))'), '(Request ID: 4d77dcf6-6dee-4a27-abc1-344c68224f5c)')
Processed batch 242, updated graph stats: None
Error occurred: (MaxRetryError('HTTPConnectionPool(host=\'trac-crimson.ern.nps.edu\', port=8090): Max retries exceeded with url: / (Caused by NameResolutionError("<

Traceback (most recent call last):
  File "c:\Users\jonathan.kasprisin\AppData\Local\miniconda3\envs\kg_ilp\Lib\site-packages\urllib3\connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jonathan.kasprisin\AppData\Local\miniconda3\envs\kg_ilp\Lib\site-packages\urllib3\util\connection.py", line 60, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jonathan.kasprisin\AppData\Local\miniconda3\envs\kg_ilp\Lib\socket.py", line 976, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
socket.gaierror: [Errno 11001] getaddrinfo failed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\jonathan

Error occurred: (MaxRetryError('HTTPConnectionPool(host=\'trac-crimson.ern.nps.edu\', port=8090): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x000001C3A7A8F170>: Failed to resolve \'trac-crimson.ern.nps.edu\' ([Errno 11001] getaddrinfo failed)"))'), '(Request ID: 8d6256ff-a4ce-46db-93f2-d36ca0845972)')
Processed batch 248, updated graph stats: None
Error occurred: (MaxRetryError('HTTPConnectionPool(host=\'trac-crimson.ern.nps.edu\', port=8090): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x000001C3A7A8F590>: Failed to resolve \'trac-crimson.ern.nps.edu\' ([Errno 11001] getaddrinfo failed)"))'), '(Request ID: 88eb552c-4d66-44c7-a0d6-5d25a03e9cfc)')
Processed batch 249, updated graph stats: None
Error occurred: (MaxRetryError('HTTPConnectionPool(host=\'trac-crimson.ern.nps.edu\', port=8090): Max retries exceeded with url: / (Caused by NameResolutionError("<

### Redload and continue buidling graph
Processed batch 236, updated graph stats: ({'Number of Nodes': 3030, 'Number of Edges': 9004,

In [22]:
# Initialize variables
cur_batch = 236

g_graphml_existing = "C:\\Users\\jonathan.kasprisin\\github\\Learning\\KG_ilp\\data_w_refine\\final_augmented_graph.graphml"
failed_batches_path = "C:\\Users\\jonathan.kasprisin\\github\\Learning\\KG_ilp\\data_w_refine\\failed_batches.pkl"
node_embd_path = "C:\\Users\\jonathan.kasprisin\\github\\Learning\\KG_ilp\\data_w_refine\\embeddings.pkl"
new_dir = "C:\\Users\\jonathan.kasprisin\\github\\Learning\\KG_ilp\\data_w_refine2"

G_existing = None
failed_batches = []
existing_node_embeddings = {}

if os.path.exists(failed_batches_path):
    with open(failed_batches_path, 'rb') as f:
        failed_batches = pickle.load(f)

#existing embeddings is a NoneType so just regenerating them
# if os.path.exists(node_embd_path):
#     with open(node_embd_path, 'rb') as f:
#         existing_node_embeddings = pickle.load(f)
# else:
#     print("No existing node embeddings file found.")


# Process documents in chunks
chunk_size = 1500
batch_size = 10

# Split all_docs into batches of size batch_size
doc_batches = [all_docs[i:i + batch_size] for i in range(0, len(all_docs), batch_size)]

#load existing graph and embeddings
G_existing = nx.read_graphml(g_graphml_existing)

#if existing_node_embeddings is nonetype use regenerate_embeddings
if not existing_node_embeddings:
    for node in tqdm(G_existing.nodes(), desc="Generating embeddings for existing nodes..."):
        existing_node_embeddings[node]= embd.embed_query(node)
print(f"Existing graph loaded with {len(G_existing.nodes())} nodes and {len(G_existing.edges())} edges.")
print(f"Num failed_batches: {len(failed_batches)}")
print(f"Num existing_node_embeddings: {len(existing_node_embeddings)}")


Generating embeddings for existing nodes...: 100%|██████████| 3030/3030 [22:55<00:00,  2.20it/s]

Existing graph loaded with 3030 nodes and 9004 edges.
Num failed_batches: 0
Num existing_node_embeddings: 3030


In [23]:
output_directory = 'data_w_refine2'

start_batch = cur_batch + 1
for batch_idx, doc_batch in tqdm(enumerate(doc_batches[start_batch:]), total=len(doc_batches[start_batch:])):
    try:
        G_existing, existing_node_embeddings, res = add_new_subgraph_from_docs(
            input_docs=doc_batch,
            llm=llm,
            embd=embd,
            data_dir_output=f"./{output_directory}/",
            verbatim=False,
            size_threshold=10,
            chunk_size=chunk_size,
            do_Louvain_on_new_graph=True,
            include_contextual_proximity=False,
            repeat_refine=2,
            similarity_threshold=0.95,
            do_simplify_graph=True,
            return_only_giant_component=False,
            save_common_graph=False,
            G_exisiting=G_existing,
            graph_GraphML_exisiting=None,
            existing_node_embeddings=existing_node_embeddings
        )

        print(f"Processed batch {batch_idx}, updated graph stats:", res)
        with open(f'{output_directory}/embeddings.pkl', 'wb') as f:
            pickle.dump(existing_node_embeddings, f)
        with open(f'{output_directory}/failed_batches.pkl', 'wb') as f:
            pickle.dump(failed_batches, f)
    except Exception as e:
        # Log the failed batch index
        failed_batches.append(batch_idx)
        print(f"Error processing batch {batch_idx} with batch size {batch_size}: {e}")

# Final graph statistics and saving
print("Final graph statistics:", res if 'res' in locals() else "No successful batches")
print("Failed batch indices:", failed_batches)

  7%|▋         | 1/14 [1:05:07<14:06:32, 3907.11s/it]

Processed batch 0, updated graph stats: ({'Number of Nodes': 3049, 'Number of Edges': 9074, 'Average Degree': 5.952115447687767, 'Density': 0.001952793782049792, 'Connected Components': 1, 'Number of Communities': 29}, False)


 14%|█▍        | 2/14 [2:06:23<12:34:17, 3771.48s/it]

Processed batch 1, updated graph stats: ({'Number of Nodes': 3075, 'Number of Edges': 9158, 'Average Degree': 5.956422764227642, 'Density': 0.001937678192657008, 'Connected Components': 1, 'Number of Communities': 25}, False)


 21%|██▏       | 3/14 [3:06:05<11:15:34, 3684.97s/it]

...

Processed batch 10, updated graph stats: ({'Number of Nodes': 3281, 'Number of Edges': 9940, 'Average Degree': 6.059128314538251, 'Density': 0.0018472952178470276, 'Connected Components': 1, 'Number of Communities': 25}, False)


Traceback (most recent call last):
  File "c:\Users\jonathan.kasprisin\AppData\Local\miniconda3\envs\kg_ilp\Lib\site-packages\urllib3\connectionpool.py", line 787, in urlopen
    response = self._make_request(
               ^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jonathan.kasprisin\AppData\Local\miniconda3\envs\kg_ilp\Lib\site-packages\urllib3\connectionpool.py", line 534, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "c:\Users\jonathan.kasprisin\AppData\Local\miniconda3\envs\kg_ilp\Lib\site-packages\urllib3\connection.py", line 516, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jonathan.kasprisin\AppData\Local\miniconda3\envs\kg_ilp\Lib\http\client.py", line 1428, in getresponse
    response.begin()
  File "c:\Users\jonathan.kasprisin\AppData\Local\miniconda3\envs\kg_ilp\Lib\http\client.py", line 331, in begin
    version, status, reason = self._read_status()
       

Error occurred: (ProtocolError('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)), '(Request ID: ec1dd24c-5cf3-4677-8a42-92651c77f5a7)')
Processed batch 11, updated graph stats: None


Traceback (most recent call last):
  File "c:\Users\jonathan.kasprisin\AppData\Local\miniconda3\envs\kg_ilp\Lib\site-packages\urllib3\connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jonathan.kasprisin\AppData\Local\miniconda3\envs\kg_ilp\Lib\site-packages\urllib3\util\connection.py", line 60, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jonathan.kasprisin\AppData\Local\miniconda3\envs\kg_ilp\Lib\socket.py", line 976, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
socket.gaierror: [Errno 11001] getaddrinfo failed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\jonathan

Error occurred: (MaxRetryError('HTTPConnectionPool(host=\'trac-crimson.ern.nps.edu\', port=8090): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x000001C3A86D1100>: Failed to resolve \'trac-crimson.ern.nps.edu\' ([Errno 11001] getaddrinfo failed)"))'), '(Request ID: da911d63-2c35-42b4-852d-0fbb8cdd6929)')
Processed batch 12, updated graph stats: None
Error occurred: (MaxRetryError('HTTPConnectionPool(host=\'trac-crimson.ern.nps.edu\', port=8090): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x000001C3A86D3320>: Failed to resolve \'trac-crimson.ern.nps.edu\' ([Errno 11001] getaddrinfo failed)"))'), '(Request ID: 2c92612e-acaf-4aa7-b30f-fdaf8f572d60)')
Processed batch 13, updated graph stats: None
Final graph statistics: None
Failed batch indices: []


In [24]:
# Initialize variables
cur_batch = cur_batch+ 10

g_graphml_existing = "C:\\Users\\jonathan.kasprisin\\github\\Learning\\KG_ilp\\data_w_refine2\\final_augmented_graph.graphml"
failed_batches_path = "C:\\Users\\jonathan.kasprisin\\github\\Learning\\KG_ilp\\data_w_refine2\\failed_batches.pkl"
node_embd_path = "C:\\Users\\jonathan.kasprisin\\github\\Learning\\KG_ilp\\data_w_refine2\\embeddings.pkl"
new_dir = "C:\\Users\\jonathan.kasprisin\\github\\Learning\\KG_ilp\\data_w_refine2"

G_existing = None
failed_batches = []
existing_node_embeddings = {}

if os.path.exists(failed_batches_path):
    with open(failed_batches_path, 'rb') as f:
        failed_batches = pickle.load(f)

#existing embeddings is a NoneType so just regenerating them
# if os.path.exists(node_embd_path):
#     with open(node_embd_path, 'rb') as f:
#         existing_node_embeddings = pickle.load(f)
# else:
#     print("No existing node embeddings file found.")


# Process documents in chunks
chunk_size = 1500
batch_size = 10

# Split all_docs into batches of size batch_size
doc_batches = [all_docs[i:i + batch_size] for i in range(0, len(all_docs), batch_size)]

#load existing graph and embeddings
G_existing = nx.read_graphml(g_graphml_existing)

#if existing_node_embeddings is nonetype use regenerate_embeddings
if not existing_node_embeddings:
    for node in tqdm(G_existing.nodes(), desc="Generating embeddings for existing nodes..."):
        existing_node_embeddings[node]= embd.embed_query(node)
print(f"Existing graph loaded with {len(G_existing.nodes())} nodes and {len(G_existing.edges())} edges.")
print(f"Num failed_batches: {len(failed_batches)}")
print(f"Num existing_node_embeddings: {len(existing_node_embeddings)}")


Generating embeddings for existing nodes...: 100%|██████████| 3281/3281 [53:46<00:00,  1.02it/s] 

Existing graph loaded with 3281 nodes and 9940 edges.
Num failed_batches: 0
Num existing_node_embeddings: 3281


In [25]:
output_directory = 'data_w_refine2'

start_batch = cur_batch + 1
for batch_idx, doc_batch in tqdm(enumerate(doc_batches[start_batch:]), total=len(doc_batches[start_batch:])):
    try:
        G_existing, existing_node_embeddings, res = add_new_subgraph_from_docs(
            input_docs=doc_batch,
            llm=llm,
            embd=embd,
            data_dir_output=f"./{output_directory}/",
            verbatim=False,
            size_threshold=10,
            chunk_size=chunk_size,
            do_Louvain_on_new_graph=True,
            include_contextual_proximity=False,
            repeat_refine=2,
            similarity_threshold=0.95,
            do_simplify_graph=True,
            return_only_giant_component=False,
            save_common_graph=False,
            G_exisiting=G_existing,
            graph_GraphML_exisiting=None,
            existing_node_embeddings=existing_node_embeddings
        )

        print(f"Processed batch {batch_idx}, updated graph stats:", res)
        with open(f'{output_directory}/embeddings.pkl', 'wb') as f:
            pickle.dump(existing_node_embeddings, f)
        with open(f'{output_directory}/failed_batches.pkl', 'wb') as f:
            pickle.dump(failed_batches, f)
    except Exception as e:
        # Log the failed batch index
        failed_batches.append(batch_idx)
        print(f"Error processing batch {batch_idx} with batch size {batch_size}: {e}")

# Final graph statistics and saving
print("Final graph statistics:", res if 'res' in locals() else "No successful batches")
print("Failed batch indices:", failed_batches)

  0%|          | 0/4 [00:00<?, ?it/s]

Processed batch 0, updated graph stats: ({'Number of Nodes': 3307, 'Number of Edges': 10027, 'Average Degree': 6.064106440882975, 'Density': 0.0018342729706240096, 'Connected Components': 1, 'Number of Communities': 28}, False)


 50%|█████     | 2/4 [1:54:57<1:56:23, 3491.73s/it]

Processed batch 1, updated graph stats: ({'Number of Nodes': 3339, 'Number of Edges': 10109, 'Average Degree': 6.055106319257263, 'Density': 0.0018139923065480115, 'Connected Components': 1, 'Number of Communities': 24}, False)


 75%|███████▌  | 3/4 [3:55:20<1:26:35, 5195.48s/it]

Processed batch 2, updated graph stats: ({'Number of Nodes': 3427, 'Number of Edges': 10500, 'Average Degree': 6.127808578932011, 'Density': 0.0017886189664133132, 'Connected Components': 1, 'Number of Communities': 23}, False)


100%|██████████| 4/4 [4:07:36<00:00, 3434.96s/it]  

Processed batch 3, updated graph stats: ({'Number of Nodes': 3431, 'Number of Edges': 10537, 'Average Degree': 6.142232585252113, 'Density': 0.0017907383630472633, 'Connected Components': 1, 'Number of Communities': 24}, False)


100%|██████████| 4/4 [4:07:36<00:00, 3714.25s/it]

Final graph statistics: ({'Number of Nodes': 3431, 'Number of Edges': 10537, 'Average Degree': 6.142232585252113, 'Density': 0.0017907383630472633, 'Connected Components': 1, 'Number of Communities': 24}, False)
Failed batch indices: []
